---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    readonly/train.csv - the training set (all tickets issued 2004-2011)
    readonly/test.csv - the test set (all tickets issued 2012-2016)
    readonly/addresses.csv & readonly/latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `readonly/train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `readonly/test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32
       
### Hints

* Make sure your code is working before submitting it to the autograder.

* Print out your result to see whether there is anything weird (e.g., all probabilities are the same).

* Generally the total runtime should be less than 10 mins. You should NOT use Neural Network related classifiers (e.g., MLPClassifier) in this question. 

* Try to avoid global variables. If you have other functions besides blight_model, you should move those functions inside the scope of blight_model.

* Refer to the pinned threads in Week 4's discussion forum when there is something you could not figure it out.

In [1]:

def blight_model():

    import pandas as pd
    import numpy as np
    from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier
    from sklearn.model_selection import train_test_split
    import sklearn.metrics
    from sklearn.preprocessing import LabelEncoder
    import csv
    from sklearn.metrics import roc_curve
    from sklearn.metrics import roc_auc_score
    from sklearn import metrics
    
    

    probar=pd.DataFrame(pd.read_csv('test.csv',encoding='cp1252'))
    
    X=pd.DataFrame(pd.read_csv ('train.csv',encoding='cp1252'))
    
    labelencoder = LabelEncoder()
    
    cero=pd.DataFrame(probar.loc[:,'ticket_id'],columns=['ticket_id'])
    
    diecinueve=probar.loc[:,'fine_amount']
    veinte=probar.loc[:,'admin_fee']
    veintiuno=probar.loc[:,'state_fee']
    veintidos=probar.loc[:,'late_fee']
    veintitres=probar.loc[:,'discount_amount']
    veinticuatro=probar.loc[:,'clean_up_cost']
    veinticinco=probar.loc[:,'judgment_amount']
    veintiseis=probar.loc[:,'disposition']
    veintisiete=probar.loc[:,'violation_code']

    
    cero.loc[:,'fine_amount']=diecinueve
    cero.loc[:,'admin_fee']=veinte
    cero.loc[:,'state_fee']=veintiuno
    cero.loc[:,'late_fee']=veintidos
    cero.loc[:,'discount_amount']=veintitres
    cero.loc[:,'clean_up_cost']=veinticuatro
    cero.loc[:,'judgement_amount']=veinticinco
    
    veintiseis_2 = labelencoder.fit_transform(probar['disposition'])
    veintisiete_2 = labelencoder.fit_transform(probar['violation_code'])
    
    cero.loc[:,'disposition']=veintiseis_2
    cero.loc[:,'violation_code']=veintisiete_2
    
    
    
    
    
    capado=X.loc[:,'compliance']!=20
    X2=X.loc[capado]
    
    X2['compliance']=X2['compliance'].replace([5,10],[0,1])
    #probar['grafitti_status']=probar['grafitti_status'].replace(['GRAFFITI TICKET'],[0])
    
    X2_T=X2.loc[:,['ticket_id','fine_amount','admin_fee','state_fee','late_fee',
                   'discount_amount','clean_up_cost','judgement_amount','disposition',
                   'violation_code','compliance']]
    
    X3_T = X2_T.dropna(subset=['compliance'])
    

    
    
    X3_T.loc[:,'ticket_id']=pd.to_numeric(X3_T.loc[:,'ticket_id'],errors='coerce')
    X3_T.loc[:,'fine_amount']=pd.to_numeric(X3_T.loc[:,'fine_amount'],errors='coerce')
    X3_T.loc[:,'admin_fee']=pd.to_numeric(X3_T.loc[:,'admin_fee'],errors='coerce')
    X3_T.loc[:,'state_fee']=pd.to_numeric(X3_T.loc[:,'state_fee'],errors='coerce')
    X3_T.loc[:,'late_fee']=pd.to_numeric(X3_T.loc[:,'late_fee'],errors='coerce')
    X3_T.loc[:,'discount_amount']=pd.to_numeric(X3_T.loc[:,'discount_amount'],errors='coerce')
    X3_T.loc[:,'clean_up_cost']=pd.to_numeric(X3_T.loc[:,'clean_up_cost'],errors='coerce')
    X3_T.loc[:,'judgement_amount']=pd.to_numeric(X3_T.loc[:,'judgement_amount'],errors='coerce')
    X3_T.loc[:,'disposition']=pd.to_numeric(X3_T.loc[:,'disposition'],errors='coerce')
    X3_T.loc[:,'violation_code']=pd.to_numeric(X3_T.loc[:,'violation_code'],errors='coerce')
    
    y=X3_T.iloc[:,[0,10]]             
                
    X4_T=X3_T.iloc[:,0:10]
    

    
    
    
    
    X_train, X_test, y_train, y_test = train_test_split(X4_T, y,random_state=0)
    
    
    
    X_train=np.nan_to_num(X_train)
    X_test=np.nan_to_num(X_test)
    
    
    
    ky=GradientBoostingClassifier(n_estimators=250,learning_rate=0.1,max_depth=13).fit(X_train,y_train.iloc[:,1]) 
    
    y_predict = ky.predict_proba(X_test)[:,1]
    
    fpr, tpr,_ = roc_curve(y_test.iloc[:,1],y_predict,pos_label=1)
    
    kk=roc_auc_score(y_test.iloc[:,1],y_predict)
    
    
    salida = ky.predict_proba(cero)[:, 1]
    salida=salida.reshape(-1,1)
    salida=pd.DataFrame(salida,columns=[''])
    

    salida_final=probar.iloc[:,0]
    salida_final=pd.DataFrame(salida_final,columns=['ticket_id'])
    

    results = pd.concat([salida_final, salida], axis=1,)
    results.set_index('ticket_id',inplace=True)
    
    #result=result.values
    print('AUC_Score: {:.2f}'.format(kk))

    return(results)

    
    
blight_model()




/opt/conda/lib/python3.6/site-packages/ipykernel/__main__.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


AUC_Score: 0.82


,
ticket_id,
284932,0.082857
285362,0.074263
285361,0.996613
285338,0.169638
285346,0.996613
285345,0.169638
285347,0.996613
285342,0.030467
285530,0.074286
